In [53]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import requests
import json
import re
from multiprocessing.dummy import Pool
import random
import time
import math
from lxml import etree
import gc

In [111]:
#搜索页面
init_search = 'https://search.suning.com/emall/searchProductList.do?pg=01&st=8&keyword={}&cp={}&paging={}'

#st=8是按销量排序
#cp是从0开始的页数后面paging=0是第一页，=1是第二页
headers={
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'zh-CN,zh;q=0.9',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.62 Safari/537.36'
        }


init_shopUrl = 'https://shop.suning.com/{}/index.html'

In [55]:
#商品页面
init_itemUrl = 'https://item.jd.com/{}.html'



init_commentUrl = 'https://sclub.jd.com/comment/productPageComments.action?productId={}&score=0&sortType=5&page={}&pageSize=10&isShadowSku=0&fold=1'
init_priceUrl = 'https://p.3.cn/prices/mgets?skuIds=J_{}'

In [80]:
def openlink(link):
    maxTryNum = 10
    for tries in range(0,maxTryNum):
        try:
            res = requests.get(link, headers = headers)
            return res
        except:
            if tries < (maxTryNum - 1):
                print("啊呀")
                continue
            else:
                print("Has tried %d times to access url %s, all failed!", maxTryNum, link)
                break

In [90]:
def getitemUrl(target,maxpage = 1):
    searchList = []
    for cp in range(0,maxpage):
        #print('cp =%d'%cp)
        searchList.append(init_search.format(target,cp,0))
        searchList.append(init_search.format(target,cp,1))
    itemList = []
    for searchPage in searchList:
        res = openlink(searchPage)
        selector = etree.HTML(res.text)
        itemList.extend(selector.xpath('//p[@class="sell-point"]/a/@href'))
    itemUrl = ['https:'+x for x in  itemList]
    print('商品列表获取完成，共有%d个商品'%len(itemUrl))
    return itemUrl

In [91]:
target = '行车记录仪'
maxpage = 3

In [92]:
itemUrl = getitemUrl(target,maxpage=3)

商品列表获取完成，共有180个商品


In [93]:
itemUrl

['http://product.suning.com/0000000000/945037073.html',
 'http://product.suning.com/0000000000/176716811.html',
 'http://product.suning.com/0000000000/178487682.html',
 'http://product.suning.com/0000000000/600397922.html',
 'http://product.suning.com/0070084946/10584689115.html',
 'http://product.suning.com/0000000000/10073327418.html',
 'http://product.suning.com/0000000000/10248934580.html',
 'http://product.suning.com/0000000000/10366391822.html',
 'http://product.suning.com/0000000000/10407142282.html',
 'http://product.suning.com/0000000000/10574033162.html',
 'http://product.suning.com/0000000000/178489722.html',
 'http://product.suning.com/0000000000/826112221.html',
 'http://product.suning.com/0000000000/612625463.html',
 'http://product.suning.com/0070110574/184530522.html',
 'http://product.suning.com/0070110574/181059127.html',
 'http://product.suning.com/0070110574/139778511.html',
 'http://product.suning.com/0000000000/191938678.html',
 'http://product.suning.com/00000000

In [126]:
res = requests.get('http://product.suning.com/0000000000/945038165.html')
selector = etree.HTML(res.text)

In [127]:
res.url

'https://product.suning.com/0000000000/945038165.html'

In [128]:
itemID = re.findall('/(\d+).html',res.url)[0]

In [129]:
shopId = re.findall('.com/(\d+)/',res.url)[0]

In [130]:
if len(shopId.lstrip('0')):
    shopUrl = init_shopUrl.format(shopId.lstrip('0'))
else :
    shopUrl = '苏宁自营'

In [135]:
title = selector.xpath('//div[@class="proinfo-title"]/h2/text()')[0].replace('\r\n','').replace('\t','')

In [136]:
itemDetail = '没有'

In [139]:
selector.xpath('/')

[]

In [170]:
res = openlink(searchPage)
selector = etree.HTML(res.text)

In [171]:
res.url

'https://search.suning.com/emall/searchProductList.do?pg=01&st=8&keyword=%E8%A1%8C%E8%BD%A6%E8%AE%B0%E5%BD%95%E4%BB%AA&cp=2&paging=1'

In [209]:
body = selector.xpath('//p[@class="com-cnt"]/em/text()')

In [6]:
def getitemInfo(itemid):
    itemLink = init_itemUrl.format(itemid)
    res = openlink(itemLink)
    print(res.url)
    selector = etree.HTML(res.text)

    title = "".join(selector.xpath(
                    '//div[@class="sku-name"]/text()')).replace(' ', '').replace('\r', '').replace('\n', '')

    itemDetail = ";".join(selector.xpath('//div[@class="p-parameter"]/ul[@class="parameter2 p-parameter-list"]/li/text()'))

    content = json.loads(requests.get(init_priceUrl.format(itemid)).text)
    price = content[0]['p']

    comment = json.loads(requests.get(init_commentUrl.format(itemid,0)).text)
    score = comment['productCommentSummary']['goodRate']
    totalComment= comment['productCommentSummary']['commentCount']
    good = comment['productCommentSummary']['goodCount']
    normal = comment['productCommentSummary']['generalCount']
    bad = comment['productCommentSummary']['poorCount']
  
    shopUrl = selector.xpath('//div[@class="name"]/a/@href')
    shopId = 'NaN'
    itemInfo = pd.DataFrame(dict(itemLink = [itemLink],
                                itemID = [itemid],
                                title = [title],
                                itemDetail = [itemDetail],
                                price = [price],
                                score = [score],
                                totalComment = [totalComment],
                                 good = [good],
                                 normal = [normal],
                                 bad = [bad],
                                 shopUrl = [shopUrl],
                                 shopId = [shopId],
                                source = 'jd'),
                                columns=['source',
                                        'itemID',
                                        'itemLink',
                                        'title',
                                        'price' ,
                                        'totalComment',
                                         'score',
                                        'good',
                                        'normal',
                                        'bad',
                                        'itemDetail',
                                         'shopId',
                                         'shopUrl'])
    print('#',end = '')
    return itemInfo
    

## 测试代码

In [9]:
target = 23931
itemidList = getitemidList(target,5)

页面获取完成，共有5页300个商品
300/300所有分类商品获取完成，共有1104个商品


### 并行爬取商品信息

In [13]:
pool = Pool(8)
time1 =time.time()
itemInfo = pool.map(getitemInfo,itemidList)
pool.close()
pool.join()
time2 =time.time()
print(str(time2-time1))

https://item.jd.com/28558124871.html
https://item.jd.com/1552906698.html
https://item.jd.com/31335145129.html
https://item.jd.com/29669188014.html
https://item.jd.com/5724670.html
#https://item.jd.com/5535583.html
##https://item.jd.com/11266666786.html
#https://item.jd.com/26003094881.html
https://item.jd.com/28558124875.html
##https://item.jd.com/1552906700.html
#https://item.jd.com/4126205.html
https://item.jd.com/29669188015.html
##https://item.jd.com/8035395.html
https://item.jd.com/29178661798.html
#https://item.jd.com/28984546116.html
#https://item.jd.com/26003094882.html
##https://item.jd.com/11266666785.html
https://item.jd.com/1552906702.html
####https://item.jd.com/29669188016.html
https://item.jd.com/11266709734.html#
https://item.jd.com/28984546117.html
https://item.jd.com/18014831570.html
##https://item.jd.com/7131530.html
https://item.jd.com/26003094883.html
#https://item.jd.com/1552906704.html
https://item.jd.com/16888252843.html
###https://item.jd.com/29669188017.html
h

https://item.jd.com/29621004169.html
https://item.jd.com/1544111653.html
#https://item.jd.com/30833211137.html
#https://item.jd.com/8692391.html
#####https://item.jd.com/5907369.html
https://item.jd.com/30513901315.html
https://item.jd.com/13764761942.html
https://item.jd.com/30833211138.html
##https://item.jd.com/1193986844.html
https://item.jd.com/1544111658.html
###https://item.jd.com/27261129714.html
https://item.jd.com/30513901316.html
https://item.jd.com/5907341.html
https://item.jd.com/8692393.html
https://item.jd.com/30833211139.html
##https://item.jd.com/1193986845.html
#https://item.jd.com/13764761941.html
###https://item.jd.com/30513901317.html
https://item.jd.com/1544111636.html
https://item.jd.com/27261129715.html
#https://item.jd.com/30833211140.html
https://item.jd.com/11862416364.html
##https://item.jd.com/6885554.html
###https://item.jd.com/1193986846.html
#https://item.jd.com/1544111641.html
https://item.jd.com/13764761943.html
https://item.jd.com/31335145128.html
htt

#https://item.jd.com/27409139653.html
#https://item.jd.com/26015254934.html
https://item.jd.com/10036764821.html
#https://item.jd.com/10347854986.html
#https://item.jd.com/11901016250.html
#https://item.jd.com/6527078.html
##https://item.jd.com/29092205044.html
###https://item.jd.com/19268080954.html
#https://item.jd.com/27644769676.html
https://item.jd.com/1790939138.html
https://item.jd.com/14929012797.html
https://item.jd.com/11901016252.html
https://item.jd.com/1199252409.html
https://item.jd.com/10347854988.html
######https://item.jd.com/29710589744.html
#https://item.jd.com/12396300673.html
https://item.jd.com/19268080955.html
https://item.jd.com/10347854992.html
https://item.jd.com/1602752291.html
https://item.jd.com/14929012798.html
https://item.jd.com/11901016249.html
https://item.jd.com/1790939139.html
########https://item.jd.com/19268080956.html
https://item.jd.com/11901016252.html
https://item.jd.com/14929012799.html
https://item.jd.com/12553462815.html
https://item.jd.com/

###https://item.jd.com/30467717996.html
##https://item.jd.com/30377827968.html
#https://item.jd.com/28655696298.html
#https://item.jd.com/14947035568.html
https://item.jd.com/27719192977.html
#https://item.jd.com/27860944795.html
#https://item.jd.com/28705769033.html
https://item.jd.com/13108241480.html
###https://item.jd.com/30889429663.html
https://item.jd.com/28655696299.html
https://item.jd.com/27719192978.html
##https://item.jd.com/30377827970.html
https://item.jd.com/27860944796.html
###https://item.jd.com/14947035564.html
##https://item.jd.com/24230892450.html
https://item.jd.com/11110979516.html
https://item.jd.com/10239827024.html
##https://item.jd.com/27860944797.html
https://item.jd.com/28655696300.html
#https://item.jd.com/13108241477.html
https://item.jd.com/30377827968.html
#https://item.jd.com/14947035565.html
#https://item.jd.com/11110979518.html
####https://item.jd.com/30982465760.html
https://item.jd.com/10239827026.html
https://item.jd.com/25224741005.html
#https://i

https://item.jd.com/27650189298.html
https://item.jd.com/1610809066.html
#https://item.jd.com/13068416903.html
#https://item.jd.com/10470670460.html
###https://item.jd.com/23789307934.html
https://item.jd.com/11524023744.html
##https://item.jd.com/26610832161.html
#https://item.jd.com/27650189299.html
https://item.jd.com/13068416901.html
https://item.jd.com/1610809067.html
https://item.jd.com/30530365657.html
https://item.jd.com/23789307936.html
#https://item.jd.com/10477505330.html
#######https://item.jd.com/11524023741.html
https://item.jd.com/20986852405.html
https://item.jd.com/11609292370.html
https://item.jd.com/30879774772.html
https://item.jd.com/10122620916.html
#https://item.jd.com/10477505331.html
https://item.jd.com/23789307944.html
#####https://item.jd.com/20986852406.html
https://item.jd.com/11524023742.html
#https://item.jd.com/11499232145.html
https://item.jd.com/26610832162.html
https://item.jd.com/30879781026.html
https://item.jd.com/13292182642.html
#https://item.jd.

https://item.jd.com/26610832158.html
https://item.jd.com/30879781032.html
###https://item.jd.com/30879781035.html
https://item.jd.com/26610832159.html
##https://item.jd.com/30282477962.html
https://item.jd.com/26610832160.html
##119.53218531608582


In [14]:
info = pd.concat(itemInfo,ignore_index=True)

In [15]:
info.to_csv('output/jd行车记录仪-info-'+str(time.strftime("%Y-%m-%d"))+'.csv',encoding='utf-8')

count       1104
unique       281
top       399.00
freq          35
Name: price, dtype: object